# 196. Delete duplicate emails
[Problem description](https://leetcode.com/problems/delete-duplicate-emails/description/)

In [1]:
import sys
sys.version

'3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]'

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]')\
    .appName('LeetCode-196-PySpark')\
    .getOrCreate()

In [3]:
# Data 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
schema = StructType([
    StructField('id', IntegerType(), False),
    StructField('email', StringType(), False)
])

person = spark.createDataFrame([[1, 'john@example.com'], [2, 'bob@example.com'], [3, 'john@example.com']], schema=schema)
person.show()

+---+----------------+
| id|           email|
+---+----------------+
|  1|john@example.com|
|  2| bob@example.com|
|  3|john@example.com|
+---+----------------+



**Write a solution to delete all duplicate emails, keeping only one unique email with the smallest id.**

In [18]:
# Solution
from pyspark.sql.functions import col, when

duplicates = person.groupBy('email').count().filter(col('count') > 1).select('email').collect()
duplicates = list(map(lambda i: i['email'], duplicates))
print(duplicates)

['john@example.com']


In [33]:
from pyspark.sql import Window
from pyspark.sql.functions import first_value

# df = person.withColumn('duplicateEmail', when(col('email').isin(duplicates), 1).otherwise(0))

window = Window.partitionBy('email').orderBy('id')
df = person.withColumn('orig', first_value(col('id')).over(window))
df.filter(df.id == df.orig).drop('orig').show()

+---+----------------+
| id|           email|
+---+----------------+
|  2| bob@example.com|
|  1|john@example.com|
+---+----------------+

